In [ ]:
import os
os.chdir('/nfs/homedirs/ayle/guided-research/SNIP-it/')

In [ ]:
import torch
from torchvision import datasets, transforms
import foolbox as fb
from experiments.main import load_checkpoint
from models import GeneralModel
from models.statistics.Metrics import Metrics
from utils.config_utils import *
from utils.model_utils import *
from utils.system_utils import *
from utils.attacks_utils import get_attack
from torch.utils.data.dataset import Dataset

In [ ]:
arguments = dict({
'eval_freq': 1000,  # evaluate every n batches
    'save_freq': 1e6,  # save model every n epochs, besides before and after training
    'batch_size': 64,  # size of batches, for Imagenette 128
    'seed': 1234,  # random seed
    'max_training_minutes': 6120 , # one hour and a 45 minutes max, process killed after n minutes (after finish of epoch)
    'plot_weights_freq': 50, # plot pictures to tensorboard every n epochs
    'prune_freq': 1, # if pruning during training: how long to wait before starting
    'prune_delay': 0, # "if pruning during training: 't' from algorithm box, interval between pruning events, default=0
    'prune_to': 5,
    'epochs': 80,
    'rewind_to': 0, # rewind to this epoch if rewinding is done
    'snip_steps': 5, # 's' in algorithm box, number of pruning steps for 'rule of thumb', TODO
    'pruning_rate': 0.0 , # pruning rate passed to criterion at pruning event. however, most override this
    'growing_rate': 0.0000 , # grow back so much every epoch (for future criterions)
    'pruning_limit': 0.80,  # Prune until here, if structured in nodes, if unstructured in weights. most criterions use this instead of the pruning_rate
    'local_pruning': 1,
    'learning_rate': 2e-3,
    'grad_clip': 10,
    'grad_noise': 0 , # added gaussian noise to gradients
    'l2_reg': 5e-5 , # weight decay
    'l1_reg': 0 , # l1-norm regularisation
    'lp_reg': 0 , # lp regularisation with p < 1
    'l0_reg': 1.0 , # l0 reg lambda hyperparam
    'hoyer_reg': 0.001 , # hoyer reg lambda hyperparam
    'beta_ema': 0.999 , # l0 reg beta ema hyperparam

    'loss': 'CrossEntropy',
    'optimizer': 'ADAM',
    'model': 'LeNet5',  # ResNet not supported with structured
    'data_set': 'FASHION',
    'ood_data_set': 'MNIST',
    'ood_data_set_prune': 'MNIST',
    'prune_criterion': 'EmptyCrit',  # options: SNIP, SNIPit, SNIPitDuring, UnstructuredRandom, GRASP, HoyerSquare, IMP, // SNAPit, StructuredRandom, GateDecorators, EfficientConvNets, GroupHoyerSquare
    'train_scheme': 'DefaultTrainer' , # default: DefaultTrainer
    'attack': 'FGSM',
    'epsilon': 12,
    'eval_ood_data_sets': ['OMNIGLOT', 'MNIST', 'KMNIST', 'INV_OMNIGLOT'],
    'eval_attacks': ['FGSM', 'L2FGSM'],
    'eval_epsilons': [4, 6, 12, 24, 48],

    'device': 'cuda',
    'results_dir': "LeNet5",

    'checkpoint_name': '2021-06-15_17.54.25_model=LeNet5_dataset=FASHION_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=5_rewind-to=0_train-scheme=DefaultTrainer_seed=1234',
    'checkpoint_model': 'LeNet5_finished',

    'disable_cuda_benchmark': 1 , # speedup (disable) vs reproducibility (leave it)
    'eval': 0,
    'disable_autoconfig': 0 , # for the brave
    'preload_all_data': 0 , # load all data into ram memory for speedups
    'tuning': 0 , # splits trainset into train and validationset, omits test set

    'get_hooks': 0,
    'track_weights': 0 , # "keep statistics on the weights through training
    'disable_masking': 1 , # disable the ability to prune unstructured
    'enable_rewinding': 0 , # enable the ability to rewind to previous weights
    'outer_layer_pruning': 0, # allow to prune outer layers (unstructured) or not (structured)
    'first_layer_dense': 0,
    'random_shuffle_labels': 0  ,# run with random-label experiment from zhang et al
    'l0': 0,  # run with l0 criterion, might overwrite some other arguments
    'hoyer_square': 0, # "run in unstructured DeephoyerSquare criterion, might overwrite some other arguments
    'group_hoyer_square': 0 ,# run in unstructured Group-DeephoyerSquare criterion, might overwrite some other arguments

    'disable_histograms': 0,
    'disable_saliency': 0,
    'disable_confusion': 0,
    'disable_weightplot': 0,
    'disable_netplot': 0,
    'skip_first_plot': 0,
    'disable_activations': 0,
    
    'input_dim': [1, 28, 28],
      'output_dim': 10,
      'hidden_dim': [512],
      'N': 60000,
    
#     'input_dim': [3, 32, 32],
#       'output_dim': 10,
#       'hidden_dim': [512],
#       'N': 60000
})

In [ ]:
DATASET_PATH = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/data'

In [ ]:
metrics = Metrics()
out = metrics.log_line
metrics._batch_size = arguments['batch_size']
metrics._eval_freq = arguments['eval_freq']
set_results_dir(arguments["results_dir"])

In [ ]:
model: GeneralModel = find_right_model(
        NETWORKS_DIR, arguments['model'],
        device=arguments['device'],
        hidden_dim=arguments['hidden_dim'],
        input_dim=arguments['input_dim'],
        output_dim=arguments['output_dim'],
        is_maskable=arguments['disable_masking'],
        is_tracking_weights=arguments['track_weights'],
        is_rewindable=arguments['enable_rewinding'],
        is_growable=arguments['growing_rate'] > 0,
        outer_layer_pruning=arguments['outer_layer_pruning'],
        maintain_outer_mask_anyway=(
                                       not arguments['outer_layer_pruning']) and (
                                           "Structured" in arguments['prune_criterion']),
        l0=arguments['l0'],
        l0_reg=arguments['l0_reg'],
        N=arguments['N'],
        beta_ema=arguments['beta_ema'],
        l2_reg=arguments['l2_reg']
).to(arguments['device'])

In [ ]:
load_checkpoint(arguments, model, out)

In [ ]:
device = arguments['device']

In [ ]:
# get criterion
criterion = find_right_model(
    CRITERION_DIR, arguments['prune_criterion'],
    model=model,
    limit=arguments['pruning_limit'],
    start=0.5,
    steps=arguments['snip_steps'],
    device=arguments['device'],
    arguments=arguments
)

# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments
)

# load OOD data
_, ood_prune_loader = find_right_model(
    DATASETS, arguments['ood_data_set_prune'],
    arguments=arguments
)

# get loss function
loss = find_right_model(
    LOSS_DIR, arguments['loss'],
    device=device,
    l1_reg=arguments['l1_reg'],
    lp_reg=arguments['lp_reg'],
    l0_reg=arguments['l0_reg'],
    hoyer_reg=arguments['hoyer_reg']
)

# get optimizer
optimizer = find_right_model(
    OPTIMS, arguments['optimizer'],
    params=model.parameters(),
    lr=arguments['learning_rate'],
    weight_decay=arguments['l2_reg'] if not arguments['l0'] else 0
)

# run_name = f'_model={arguments["model"]}_dataset={arguments["data_set"]}_ood-dataset={arguments["ood_data_set"]}' + \
#            f'_attack={arguments["attack"]}_epsilon={arguments["epsilon"]}_prune-criterion={arguments["prune_criterion"]}' + \
#            f'_pruning-limit={arguments["pruning_limit"]}_prune-freq={arguments["prune_freq"]}_prune-delay={arguments["prune_delay"]}' + \
#            f'_rewind-to={arguments["rewind_to"]}_train-scheme={arguments["train_scheme"]}_seed={arguments["seed"]}'


# # build trainer
# trainer = find_right_model(
#     TRAINERS_DIR, arguments['train_scheme'],
#     model=model,
#     loss=loss,
#     optimizer=optimizer,
#     device=device,
#     arguments=arguments,
#     train_loader=train_loader,
#     test_loader=test_loader,
#     ood_loader=ood_loader,
#     ood_prune_loader=ood_prune_loader,
#     metrics=metrics,
#     criterion=criterion,
#     run_name=run_name
# )

# trainer.train()

In [ ]:
results = {}

In [ ]:
results = {'train_acc': trainer.train_acc, 'test_acc': trainer.test_acc, 'sparsity': trainer.sparsity,
               'filename': DATA_MANAGER.stamp}

In [ ]:
model.eval()

In [ ]:
for attack in arguments['eval_attacks']:
    for epsilon in arguments['eval_epsilons']:
        out("Attack {}".format(attack))
        # build tester
        tester = find_right_model(
            TESTERS_DIR, 'AdversarialEvaluation',
            attack=attack,
#             model=trainer._model,
            model = model,
            device=device,
            arguments=None,
            test_loader=test_loader,
        )

        out("Epsilon {}".format(str(epsilon)))
        res = tester.evaluate(epsilon=epsilon)

        for key, value in res.items():
            results[key] = value

In [ ]:
with torch.no_grad():
    for ood_data_set in arguments['eval_ood_data_sets']:
        out("OOD Dataset: {}".format(ood_data_set))
        # load data
        _, test_loader = find_right_model(
            DATASETS, arguments['data_set'],
            arguments=arguments
        )

        # load OOD data
        _, ood_loader = find_right_model(
            DATASETS, ood_data_set,
            arguments=arguments
        )
        # build tester
        tester = find_right_model(
            TESTERS_DIR, 'OODEvaluation',
#             model=trainer._model,
            model = model,
            device=device,
            arguments=None,
            test_loader=test_loader,
            ood_loader=ood_loader,
            ood_dataset=ood_data_set
        )
        res = tester.evaluate()

        for key, value in res.items():
            results[key] = value
            

In [ ]:
class DS(Dataset):

    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.mean = [0.485, 0.456, 0.406]  # avg 0.449
        self.std = [0.229, 0.224, 0.225]  # avg 0.226
        self.transforms = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ]
        )

    def __getitem__(self, item):
        image = self.images[item] / 255
        image = self.transforms(image.transpose((1, 2, 0)))
        return image.to(torch.float32), torch.tensor(self.labels[item], dtype=torch.float32)

    def __len__(self):
        return len(self.images)

with torch.no_grad():
    if arguments["data_set"] == "CIFAR10":
        ds_path = os.path.join(DATASET_PATH, "cifar10_corrupted")
        for ds_dataset_name in os.listdir(ds_path):
            npz_dataset = np.load(os.path.join(ds_path, ds_dataset_name))

            ds_dataset = DS(npz_dataset["images"], npz_dataset["labels"])
            ds_loader = torch.utils.data.DataLoader(
                ds_dataset,
                batch_size=arguments['batch_size'],
                shuffle=False,
                pin_memory=True,
                num_workers=4
            )

            # build tester
            tester = find_right_model(
                TESTERS_DIR, 'DSEvaluation',
                model=trainer._model,
                device=device,
                arguments=None,
                test_loader=test_loader,
                ds_loader=ds_loader,
                ds_dataset=ds_dataset_name.split('.')[0]
            )
            res = tester.evaluate()

            for key, value in res.items():
                results[key] = value

            break

In [ ]:
results